In [5]:
# !pip install statsmodels

In [1]:
##Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
# Loading dataset from Excel
df = pd.read_excel('data.xlsx')

In [3]:
df

,Unnamed: 0,Time,Temperature (C),Humidity (%),Feels Like,Pressure
0,0,2023-12-23 00:00:00,20.621781,40.165664,1.424202,1.626998
1,1,2023-12-23 01:00:00,24.655000,64.463843,0.184435,3.732457
2,2,2023-12-23 02:00:00,23.123958,61.205720,3.047822,3.248164
3,3,2023-12-23 03:00:00,22.190609,61.870215,2.513395,4.246117
4,4,2023-12-23 04:00:00,19.092130,63.138110,0.257394,3.288064
...,...,...,...,...,...,...
67,67,2023-12-25 19:00:00,23.615379,69.153462,3.459476,0.469910
68,68,2023-12-25 20:00:00,18.521855,68.873419,3.259806,2.891401
69,69,2023-12-25 21:00:00,24.908209,47.553469,1.121347,0.179711
70,70,2023-12-25 22:00:00,23.405713,54.917455,3.560896,2.327990


In [4]:
#  data has a timestamp column, set it as the index
df['Time'] = pd.to_datetime(df['Time'])
df.set_index('Time', inplace=True)

In [6]:
df.columns

Index(['Unnamed: 0', 'Temperature (C)', 'Humidity (%)', 'Feels Like',
       'Pressure'],
      dtype='object')

In [9]:
df = df.asfreq('60T')

In [10]:
df

,Unnamed: 0,Temperature (C),Humidity (%),Feels Like,Pressure
Time,,,,,
2023-12-23 00:00:00,0.0,20.621781,40.165664,1.424202,1.626998
2023-12-23 01:00:00,1.0,24.655000,64.463843,0.184435,3.732457
2023-12-23 02:00:00,2.0,23.123958,61.205720,3.047822,3.248164
2023-12-23 03:00:00,3.0,22.190609,61.870215,2.513395,4.246117
2023-12-23 04:00:00,4.0,19.092130,63.138110,0.257394,3.288064
...,...,...,...,...,...
2023-12-25 19:00:00,67.0,23.615379,69.153462,3.459476,0.469910
2023-12-25 20:00:00,68.0,18.521855,68.873419,3.259806,2.891401
2023-12-25 21:00:00,69.0,24.908209,47.553469,1.121347,0.179711


In [13]:
# Choose the target variable for prediction (e.g., 'Temperature')
target_variable = 'Temperature (C)'

# Fit ARIMA model
order = (10, 1, 2)  # Example order parameters (p, d, q)
model = ARIMA(df[target_variable], order=order)
model_fit = model.fit()

# Make predictions for the next three hours (12 time intervals if your data is in 15-minute intervals)
forecast_steps = 6
forecast = model_fit.forecast(steps=forecast_steps, typ='levels')

# Print and plot the forecast
print("ARIMA Forecast for the next three hours:")
print(forecast)

/home/zeeshan/anaconda3/envs/RlPrompt/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'


ARIMA Forecast for the next three hours:
2023-12-26 00:00:00    22.083254
2023-12-26 01:00:00    19.359128
2023-12-26 02:00:00    22.057107
2023-12-26 03:00:00    21.217537
2023-12-26 04:00:00    19.795153
2023-12-26 05:00:00    22.678550
Freq: 60T, Name: predicted_mean, dtype: float64


/home/zeeshan/anaconda3/envs/RlPrompt/lib/python3.9/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/zeeshan/anaconda3/envs/RlPrompt/lib/python3.9/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['typ']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)


In [15]:
forecast = pd.DataFrame(forecast)

In [18]:
forecast.columns = ["Predicted Temp"]

In [19]:
forecast

,Predicted Temp
2023-12-26 00:00:00,22.083254
2023-12-26 01:00:00,19.359128
2023-12-26 02:00:00,22.057107
2023-12-26 03:00:00,21.217537
2023-12-26 04:00:00,19.795153
2023-12-26 05:00:00,22.678550
